In [10]:
import random
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, HoverTool, ColumnDataSource
from bokeh.palettes import Category20
import geopandas as gpd
import json
import networkx as nx

output_file("enhanced_geospatial_network.html")
world = gpd.read_file("ne_110m_admin_0_countries.shp") 
world['dummy_name'] = [f"Country {i}" for i in range(len(world))]

# Creating a random network graph
countries = world['dummy_name'][:20]  
G = nx.Graph()
for country in countries:
    G.add_node(country, x=random.uniform(-180, 180), y=random.uniform(-90, 90))

# Adding random edges with weights
for _ in range(40): 
    node1, node2 = random.sample(list(G.nodes), 2)
    weight = random.uniform(0.5, 2.0) 
    G.add_edge(node1, node2, weight=weight)

# Extracting the node positions
node_positions = {node: (G.nodes[node]['x'], G.nodes[node]['y']) for node in G.nodes}
edges = list(G.edges)

# Edge coordinates for Bokeh
edge_xs = []
edge_ys = []
edge_weights = []
for edge in edges:
    x0, y0 = node_positions[edge[0]]
    x1, y1 = node_positions[edge[1]]
    edge_xs.append([x0, x1])
    edge_ys.append([y0, y1])
    edge_weights.append(G.edges[edge]["weight"])

# Bokeh plot
p = figure(
    title="Enhanced Geospatial Network and Interconnections",
    width=800,
    height=600,
    background_fill_color="#f5f5f5",
    toolbar_location="below"
)

p.title.text_color = "navy"
p.title.text_font_size = "18pt"
p.title.align = "center"

# Map
geojson_data = json.loads(world.to_json())
geojson_source = GeoJSONDataSource(geojson=json.dumps(geojson_data))
p.patches(
    "xs",
    "ys",
    source=geojson_source,
    fill_alpha=0.5,
    fill_color="lightblue",
    line_color="white",
    line_width=0.5
)

# Adding edges with variable thickness
p.multi_line(
    xs=edge_xs,
    ys=edge_ys,
    line_color="gray",
    line_width=edge_weights,
    alpha=0.7
)

# Adding colorful nodes
node_colors = [Category20[20][i % 20] for i in range(len(node_positions))]
node_sizes = [random.randint(10, 20) for _ in range(len(node_positions))]
node_source = ColumnDataSource(data=dict(
    x=[pos[0] for pos in node_positions.values()],
    y=[pos[1] for pos in node_positions.values()],
    name=list(node_positions.keys()),
    color=node_colors,
    size=node_sizes
))

p.circle(
    "x",
    "y",
    source=node_source,
    size="size",
    fill_color="color",
    line_color="black",
    line_width=0.5,
    alpha=0.9
)

hover = HoverTool(tooltips=[("Country", "@name")])
p.add_tools(hover)
show(p)